In [1]:
# %load_ext autoreload
# %autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import skopi as sk

import torch
from tqdm import tqdm 
import time
import h5py

from neurorient import uniform_points_on_sphere
from neurorient.reconstruction.slicing import get_real_mesh

import os
os.environ["USE_CUPY"] = "1"
os.environ.get('USE_CUPY')

/pscratch/sd/z/zhantao/conda/om/lib/python3.9/site-packages/skopi/diffraction.py:76: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_phase(atom_pos, q_xyz):
/pscratch/sd/z/zhantao/conda/om/lib/python3.9/site-packages/skopi/diffraction.py:90: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def cal(f_hkl, atom_pos, q_xyz, xyz_ind, pixel_number):
/pscratch/sd/z

'1'

In [2]:
save_dir = '/pscratch/sd/z/zhantao/neurorient_repo/data'

In [3]:
n_pixels, det_size, det_dist = (128, 0.1, 0.2)
det = sk.SimpleSquareDetector(n_pixels, det_size, det_dist)

In [4]:
pdb = '5BWK'
increase_factor = 10
num_images = 10000
poisson = True

In [5]:
# Set up x-ray beam
beam = sk.Beam("input/beam/amo86615.beam")
beam.set_photons_per_pulse(increase_factor * beam.get_photons_per_pulse())

# Set up particle
# pdb file of lidless mmCpn in open state
particle = sk.Particle()
particle.read_pdb(f"input/pdb/{pdb}.pdb", ff='WK')

# Set up SPI experiment
exp = sk.SPIExperiment(det, beam, particle)

Unknown element or wrong line: 
 HETATM41928 ZN    ZN A 500     326.577  97.436 630.933  1.00354.84          ZN  

Unknown element or wrong line: 
 HETATM41929 ZN    ZN D 501     334.439 -15.371 652.804  1.00363.55          ZN  

Unknown element or wrong line: 
 HETATM41930 ZN    ZN M 401     250.625  65.139 632.217  1.00356.85          ZN  

Unknown element or wrong line: 
 HETATM41931 ZN    ZN O 401     262.374 -47.288 653.046  1.00354.90          ZN  



/pscratch/sd/z/zhantao/conda/om/lib/python3.9/site-packages/numba/cuda/cudadrv/devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [6]:
# generate random orientations
orientations = sk.get_random_quat(num_images)
# setup experiment with generated orientations
exp.set_orientations(orientations)
# preallocate memory for images
images = np.zeros((len(orientations), ) + det.shape[1:])
# calculate and fill in images
for i in tqdm(range(len(orientations))):
    if poisson:
        images[i] = exp.generate_image_stack(return_photons=True, return_intensities=False)[0]
    else:
        images[i] = exp.generate_image_stack(return_photons=False, return_intensities=True)[0]

100%|██████████| 10000/10000 [01:33<00:00, 106.52it/s]


In [7]:
img_real_mesh = get_real_mesh(det.shape[1], det.pixel_position_reciprocal.max())
_mesh, _len = exp.det.get_reciprocal_mesh(voxel_number_1d=exp.mesh_size)
vol_real_mesh = get_real_mesh(_mesh.shape[0], _mesh.max())

In [8]:
pt_fpath = os.path.join(save_dir, f'{pdb}_increase{increase_factor:d}_poisson{poisson}_num{num_images//1000:d}K.pt')
torch.save(
    {
        'orientations': torch.from_numpy(orientations).float(),
        'intensities': torch.from_numpy(images).float(),
        'pixel_position_reciprocal': torch.from_numpy(det.pixel_position_reciprocal).float(),
        'pixel_index_map': torch.from_numpy(det.pixel_index_map).long(),
        'volume': torch.from_numpy(exp.volumes[0]).to(torch.complex64),
        'img_real_mesh': img_real_mesh.float(),
        'vol_real_mesh': vol_real_mesh.float(),
        'time_stamp': time.strftime("%Y%m%d-%H%M")
    }, pt_fpath
)
print("data wrote to: \n", pt_fpath)

data wrote to: 
 /pscratch/sd/z/zhantao/neurorient_repo/data/5BWK_increase100_poissonTrue_num10K.pt


In [9]:
h5_fpath = os.path.join(
    save_dir, 
    f'{pdb}_increase{increase_factor:d}_poisson{poisson}_num{num_images//1000}K.h5')
with h5py.File(h5_fpath, 'w') as f:
    f.create_dataset('intensities', data=images[:, None])
    f.create_dataset('orientations', data=orientations)
    f.create_dataset('pixel_position_reciprocal', data=det.pixel_position_reciprocal)
    f.create_dataset('pixel_distance_reciprocal', data=det.pixel_distance_reciprocal)
    f.create_dataset('pixel_index_map', data=det.pixel_index_map)
print("data wrote to: \n", h5_fpath)

data wrote to: 
 /pscratch/sd/z/zhantao/neurorient_repo/data/5BWK_increase100_poissonTrue_num10K.h5
